In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os

In [2]:
timeseries_path = os.path.join('..', 'pvops', 'timeseries')
sys.path.append('..')
sys.path.append(timeseries_path)

In [3]:
from pvops.timeseries import utils, preprocess
from pvops.timeseries.models import linear
from pvops.text2time import utils as t2t_utils, preprocess as t2t_preprocess

In [4]:
example_OMpath = os.path.join('example_data', 'example_om_data2.csv')
example_prodpath = os.path.join('example_data', 'example_prod_with_covariates.csv')
example_metapath = os.path.join('example_data', 'example_metadata2.csv')

In [5]:
prod_data = pd.read_csv(example_prodpath, error_bad_lines=False, engine='python')
om_data = pd.read_csv(example_OMpath, error_bad_lines=False, engine='python')
metadata = pd.read_csv(example_metapath, error_bad_lines=False, engine='python')

In [6]:
prod_data.head()

,date,randid,generated_kW,expected_kW,irrad_poa_Wm2,temp_amb_C,wind_speed_ms,temp_mod_C
0,2018-04-01 07:00:00,R15,0.475,0.527845,0.02775,16.570,4.2065,14.1270
1,2018-04-01 08:00:00,R15,1332.547,1685.979445,87.91450,16.998,4.1065,15.8610
2,2018-04-01 09:00:00,R15,6616.573,7343.981135,367.90350,20.168,4.5095,24.5745
3,2018-04-01 10:00:00,R15,8847.800,10429.876422,508.28700,21.987,4.9785,30.7740
4,2018-04-01 11:00:00,R15,11607.389,12981.228814,618.79450,23.417,4.6410,35.8695


In [7]:
#Format for dictionaries is {pvops variable: user-specific column names}
prod_col_dict = {'siteid': 'randid', 
                 'timestamp': 'date', 
                 'powerprod': 'generated_kW',
                 'irradiance':'irrad_poa_Wm2',
                 'temperature':'temp_mod_C', # Optional parameter, used by one of the modeling structures
                 'baseline': 'IEC_pstep', #user's name choice for new column (baseline expected energy defined by user or calculated based on IEC)
                 'dcsize': 'dcsize', #user's name choice for new column (System DC-size, extracted from meta-data)
                 'compared': 'Compared',#user's name choice for new column
                 'energy_pstep': 'Energy_pstep'} #user's name choice for new column

metad_col_dict = {'siteid': 'randid',
                  'dcsize': 'DC_Size_kW',
                  'COD': 'COD'}

In [8]:
prod_data_converted = t2t_preprocess.prod_date_convert(prod_data, prod_col_dict)
prod_data_datena_d, _ = t2t_preprocess.prod_nadate_process(prod_data_converted, prod_col_dict, pnadrop=True)

In [9]:
# for xcol in ['irrad_poa_Wm2', 'temp_mod_C', 'wind_speed_ms']:
#     plt.scatter(prod_data_datena_d[xcol], prod_data_datena_d[prod_col_dict['powerprod']])
#     plt.title(xcol)
#     plt.grid()
#     plt.show()

In [10]:
model_prod_data = prod_data_datena_d.dropna(subset=['irrad_poa_Wm2', 'temp_amb_C', 'wind_speed_ms']+[prod_col_dict['powerprod']])
model_prod_data = model_prod_data[model_prod_data['randid'] == 'R15']

model = linear.modeller(model_prod_data,
                prod_col_dict,
                metadata,
                metad_col_dict,
                kernel_type='polynomial',
                X_parameters=['irrad_poa_Wm2', 'temp_amb_C', 'wind_speed_ms'],
                degree=2,
                verbose=1)


Begin training.
[OLS] Mean squared error: 2932190.66
[OLS] Coefficient of determination: 0.92
[RANSAC] Mean squared error: 3031673.62
[RANSAC] Coefficient of determination: 0.92

Begin testing.
[OLS] Mean squared error: 8895298.09
[OLS] Coefficient of determination: 0.79
[RANSAC] Mean squared error: 9474790.17
[RANSAC] Coefficient of determination: 0.78


In [13]:
model.estimators

[('OLS', LinearRegression()), ('RANSAC', RANSACRegressor(random_state=42))]